In [1]:
import pandas as pd

# 1. Загружаем датасет
df = pd.read_csv("OnlineRetail.csv", encoding="latin1")  # или другой encoding, как у тебя

# 2. Базовая чистка (можешь подстроить под свой EDA)
df = df.dropna(subset=["CustomerID", "StockCode"])      # выкинуть строки без клиента/товара
df = df[df["Quantity"] > 0]                             # убрать возвраты
df = df[df["UnitPrice"] > 0]                            # убрать нулевые/отрицательные цены

# Приводим типы
df["CustomerID"] = df["CustomerID"].astype(int).astype(str)
df["StockCode"]  = df["StockCode"].astype(str)
df["InvoiceNo"]  = df["InvoiceNo"].astype(str)

print("Transactions:", len(df))
print("Unique customers:", df["CustomerID"].nunique())
print("Unique products:", df["StockCode"].nunique())


Transactions: 397884
Unique customers: 4338
Unique products: 3665


In [3]:
from tqdm import tqdm

# Для каждой транзакции (чека) берём уникальные товары
baskets = (
    df.groupby("InvoiceNo")["StockCode"]
      .apply(lambda x: sorted(set(x)))   # set - чтобы убрать дубли в одном чеке
      .tolist()
)

len(baskets), baskets[0][:5]

(18532, ['21730', '22752', '71053', '84029E', '84029G'])

In [4]:
from collections import defaultdict
from itertools import combinations

item_counts = defaultdict(int)      # сколько чеков с каждым товаром
co_counts   = defaultdict(int)      # сколько чеков с каждой парой товаров

for basket in tqdm(baskets):
    # считаем, в скольких корзинах встретился товар
    for item in set(basket):
        item_counts[item] += 1

    # пары товаров внутри корзины
    for i, j in combinations(sorted(set(basket)), 2):
        # сортируем пару, чтобы (i, j) и (j, i) не дублировались
        pair = (i, j)
        co_counts[pair] += 1


100%|██████████████████████████████████████████████████████████████████████████| 18532/18532 [00:04<00:00, 4272.72it/s]


In [5]:
neighbors = defaultdict(list)

# из co_counts делаем соседей в обе стороны
for (i, j), c in co_counts.items():
    # P(j | i)
    score_ij = c / item_counts[i]
    neighbors[i].append((j, score_ij))

    # P(i | j)
    score_ji = c / item_counts[j]
    neighbors[j].append((i, score_ji))

# теперь для каждого товара отсортируем соседей и возьмём топ-K
TOP_K = 10

product_recs = {}  # итоговый dict: product_id -> список соседей

for item, neigh_list in neighbors.items():
    # сортируем по score по убыванию
    neigh_list_sorted = sorted(neigh_list, key=lambda x: x[1], reverse=True)
    # берём топ-K и кладём в удобный формат
    product_recs[item] = [
        {"product_id": nid, "score": float(score)}
        for nid, score in neigh_list_sorted[:TOP_K]
    ]


In [6]:
# берём первое описание для каждого StockCode
product_info = (
    df[["StockCode", "Description"]]
    .drop_duplicates("StockCode")
    .set_index("StockCode")["Description"]
    .to_dict()
)

import json

output_products = {
    "products": {}
}

for pid in product_info.keys():
    output_products["products"][pid] = {
        "description": product_info.get(pid, ""),
        "neighbors": product_recs.get(pid, [])  # может быть пустой список, если мало данных
    }

with open("product_neighbors.json", "w", encoding="utf-8") as f:
    json.dump(output_products, f, ensure_ascii=False, indent=2)
